In [1]:
import requests
import time as t
import pandas as pd
from collections import Counter

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
# проверим ответ
response = requests.get('https://djinni.co/jobs/')
response

<Response [200]>

In [4]:
# зададим параметры
driver_path = "C:/Users/MaxAndriichuk/.jupyter/chromedriver"
browser = webdriver.Chrome(executable_path=driver_path)

url = 'https://djinni.co/jobs/?page='

data = pd.DataFrame()

# найдем номер последней страницы для цикла
browser.get('https://djinni.co/jobs/')
page_ = browser.find_elements_by_class_name("page-item")
pages = []
for i in page_:
    pages.append(i.text)
last_page = int(pages[-2])

# пролистываем страницы
for i in range(637, last_page+1):
    page = url+str(i)
    browser.get(page)
    t.sleep(5)
    
    # берем весь контенкт с карточек на этой странице
    full_page = []
    for i in browser.find_elements_by_class_name("list-jobs__item"): 
        full_page.append(i.text)
    
    # определяем контент карточек в словарь
    full_page_dict = {}
    date_list = []
    position_list = []
    about_list = []
    recruiting_by_list = []
    location_experiance_list = []
    
    # заполняем значения для словарей
    for i in range(len(full_page)):
        date_list.append(full_page[i].split('\n')[0])
        position_list.append(full_page[i].split('\n')[1])
        about_list.append(full_page[i].split('\n')[2])
        recruiting_by_list.append(full_page[i].split('\n')[4])
        location_experiance_list.append(full_page[i].split('\n')[5])
    
    # к каждому ключу словаря присваиваем значение
    full_page_dict['date'] = date_list
    full_page_dict['position'] = position_list
    full_page_dict['about'] = about_list
    full_page_dict['recruiting_by'] = recruiting_by_list
    full_page_dict['location_experiance'] = location_experiance_list
    # датафрейм текущей страницы
    df = pd.DataFrame.from_dict(full_page_dict)
    
    # аккумулируем инфу в датасет
    data = data.append(df)
    
browser.quit()
data.to_excel("Parsing experiment.xlsx", index=False)
data

<ipython-input-4-1207b466274a>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=driver_path)
<ipython-input-4-1207b466274a>:10: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  page_ = browser.find_elements_by_class_name("page-item")
<ipython-input-4-1207b466274a>:24: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  for i in browser.find_elements_by_class_name("list-jobs__item"):


IndexError: list index out of range

In [2]:
data[:3]

NameError: name 'data' is not defined

In [6]:
# создадим position_name и salary
data[['position_name', 'salary']] = data['position'].str.split('$', expand=True)

In [7]:
data.position_name.value_counts()[:5]

DevOps Engineer      5
Unity developer      4
Java Developer       4
DevOps               4
Node.js Developer    4
Name: position_name, dtype: int64

In [8]:
# value_list = []
# for i in list_of_position_name:
#     value = i.split(' ')
#     value_list.append(value)
    
# value_list

In [9]:
# from collections import Counter
# import json

# words = list_of_position[1].split()
# d = {' '.join(words):n for words,n in Counter(zip(words, words[1:])).items() if not  words[0][-1]==(',')}

# print(json.dumps(d, indent=4))

In [10]:
list_of_position_name = data['position_name'].tolist()
full_list = []
for i in range(len(list_of_position_name)):
    words = list_of_position_name[i].split()
    d = {' '.join(words):n for words,n in Counter(zip(words, words[1:])).items() if not  words[0][-1]==(',')}
    this_list = list(d.keys())
    for z in range(len(this_list)):
        full_list.append(this_list[z])

full_list

['Senior/Lead Backend',
 'Backend Engineer',
 'Engineer (Relocate',
 '(Relocate to',
 'to Dubai)',
 'Dubai) від',
 'Python Team',
 'Team Leader',
 'Middle/Senior Full-Stack',
 'Full-Stack Developer',
 'Developer (C#,',
 'Senior Flutter',
 'Flutter Engineer',
 'Middle QA',
 'QA Automation',
 'Automation (Linux)',
 '(Linux) Engineer',
 'Engineer Python',
 'Middle/Senior C++',
 'C++ /',
 '/ Linux',
 'Linux Developer',
 'NodeJS Developer',
 'Node.js разработчик',
 'разработчик (Senior)',
 'Senior Java',
 'Java Developer',
 'Senior Full-Stack',
 'Full-Stack Developer',
 'Developer (Java+Vue+basic',
 '(Java+Vue+basic Kotlin)',
 'Senior Frontend',
 'Frontend Developer',
 'Developer (Typescript/React)',
 'Middle SDET/QA',
 'SDET/QA Automation',
 'Fullstack Engineer',
 'Engineer (.Net)',
 'Senior Python',
 'Python Developer',
 'DevOps and',
 'and Release',
 'Release Engineer',
 'Middle/Senior Data',
 'Data QA',
 'QA Engineer',
 'Senior DevOps/SRE',
 'DevOps/SRE Engineer',
 'Blockchain dev',
 'S

In [11]:
Counter(full_list)

Counter({'Senior/Lead Backend': 1,
         'Backend Engineer': 4,
         'Engineer (Relocate': 1,
         '(Relocate to': 1,
         'to Dubai)': 1,
         'Dubai) від': 1,
         'Python Team': 4,
         'Team Leader': 1,
         'Middle/Senior Full-Stack': 1,
         'Full-Stack Developer': 5,
         'Developer (C#,': 1,
         'Senior Flutter': 1,
         'Flutter Engineer': 1,
         'Middle QA': 3,
         'QA Automation': 9,
         'Automation (Linux)': 1,
         '(Linux) Engineer': 1,
         'Engineer Python': 1,
         'Middle/Senior C++': 2,
         'C++ /': 1,
         '/ Linux': 1,
         'Linux Developer': 1,
         'NodeJS Developer': 2,
         'Node.js разработчик': 1,
         'разработчик (Senior)': 1,
         'Senior Java': 9,
         'Java Developer': 14,
         'Senior Full-Stack': 1,
         'Developer (Java+Vue+basic': 1,
         '(Java+Vue+basic Kotlin)': 1,
         'Senior Frontend': 8,
         'Frontend Developer': 4,


In [12]:
from collections import Counter
Counter_ = Counter(full_list)
df = pd.DataFrame.from_dict(Counter_, orient='index').reset_index()
df = df.rename(columns={'index':'position', 0:'count'})
df

,position,count
0,Senior/Lead Backend,1
1,Backend Engineer,4
2,Engineer (Relocate,1
3,(Relocate to,1
4,to Dubai),1
...,...,...
892,/ Site,1
893,Site manger,1
894,manger /SMM,1
895,QA (JS),1


In [13]:
df = df.sort_values('count', ascending=False)

In [ ]:
df.to_excel("Profesion_ranking.xlsx", index=False)

In [ ]:
browser.quit()

In [16]:
# обьединим скачаные датасеты (прерывания изза бана сайта)
Profesion_ranking1 = pd.read_excel('Profesion_ranking1.xlsx')
Profesion_ranking2 = pd.read_excel('Profesion_ranking2.xlsx')
Profesion_ranking3 = pd.read_excel('Profesion_ranking3.xlsx')
Profesion_ranking4 = pd.read_excel('Profesion_ranking4.xlsx')

In [25]:
Profesion_ranking = Profesion_ranking1.append(Profesion_ranking2)
Profesion_ranking = Profesion_ranking.append(Profesion_ranking3)
Profesion_ranking = Profesion_ranking.append(Profesion_ranking4)
Profesion_ranking[:3]

,position,count
0,DevOps Engineer,50
1,Software Engineer,48
2,Java Developer,45


In [40]:
aa = Profesion_ranking.groupby('position', as_index=False).agg("sum")
type(aa)

pandas.core.frame.DataFrame

In [50]:
bb = aa.sort_values('count', ascending=False)
bb

,position,count
2404,DevOps Engineer,330
4096,Java Developer,305
5721,QA Engineer,264
6598,Software Engineer,260
6331,Senior Java,230
...,...,...
4564,Logo Designer,1
4563,"Logistics Coordinator,",1
4562,Logistic and,1
4561,Locations Business,1


In [51]:
bb.to_excel("Profesion_ranking(summary).xlsx", index=False)